## Section 1: Setup and Dependencies

Install and verify all required packages. We'll use Ollama for embeddings and LLM, ChromaDB for storage, and additional libraries for advanced analysis.

In [2]:
# Install required packages
import subprocess
import sys

packages = [
    'chromadb>=0.5.0',
    'ollama',
    'numpy',
    'matplotlib',
    'scikit-learn',
    'pandas',
    'rank-bm25',
    'requests',
]

print("Installing packages...")
for package in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

print("✅ All packages installed!")
print("\n📝 Notes:")
print("- Make sure Ollama is running: ollama serve")
print("- Pull models if needed:")
print("    ollama pull nomic-embed-text (recommended)")
print("    ollama pull mistral (for LLM answer generation)")
print("    ollama pull neural-search (alternative embedding model)")

Installing packages...


You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/

✅ All packages installed!

📝 Notes:
- Make sure Ollama is running: ollama serve
- Pull models if needed:
    ollama pull nomic-embed-text (recommended)
    ollama pull mistral (for LLM answer generation)
    ollama pull neural-search (alternative embedding model)


You should consider upgrading via the '/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
# Core imports and setup
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
from dataclasses import dataclass
from datetime import datetime
import time
import json
from pathlib import Path

# For text processing
from rank_bm25 import BM25Okapi

# For embeddings
import chromadb
from chromadb.config import Settings

print("✅ All imports successful!")
print("\n📊 Ready for advanced semantic search experiments")

/Users/ankit.jha/Documents/codes/llm_playground/RAG_101/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ All imports successful!

📊 Ready for advanced semantic search experiments


In [4]:
# Ollama embedder with timing support
class TimedOllamaEmbedder:
    """Generate embeddings using Ollama with performance tracking."""
    
    def __init__(self, model: str = "nomic-embed-text", base_url: str = "http://localhost:11434"):
        self.model = model
        self.base_url = base_url.rstrip("/")
        self.endpoint = f"{self.base_url}/api/embed"
        self.timings = []
        
        # Check connection
        try:
            response = requests.get(f"{self.base_url}/api/tags", timeout=2)
            print(f"✅ Connected to Ollama at {self.base_url}")
            print(f"   Using model: {self.model}")
        except:
            print(f"❌ Cannot connect to Ollama at {self.base_url}")
            print(f"   Start with: ollama serve")
            raise
    
    def embed(self, text: str) -> Tuple[List[float], float]:
        """Generate embedding for a single text, returns (embedding, time_ms)."""
        if not text.strip():
            return [0.0] * 768, 0.0
        
        start = time.time()
        response = requests.post(
            self.endpoint,
            json={"model": self.model, "input": text.strip()},
            timeout=30
        )
        elapsed_ms = (time.time() - start) * 1000
        self.timings.append(elapsed_ms)
        
        response.raise_for_status()
        data = response.json()
        return data["embeddings"][0], elapsed_ms
    
    def embed_batch(self, texts: List[str]) -> Tuple[List[List[float]], List[float]]:
        """Generate embeddings for multiple texts."""
        embeddings = []
        timings = []
        for text in texts:
            emb, t = self.embed(text)
            embeddings.append(emb)
            timings.append(t)
        return embeddings, timings
    
    def get_performance_stats(self) -> Dict:
        """Return timing statistics."""
        if not self.timings:
            return {"count": 0}
        return {
            "count": len(self.timings),
            "mean_ms": np.mean(self.timings),
            "median_ms": np.median(self.timings),
            "min_ms": np.min(self.timings),
            "max_ms": np.max(self.timings),
            "total_ms": np.sum(self.timings),
        }

# Initialize embedder
embedder = TimedOllamaEmbedder()
print("\n✅ Embedder initialized!")

✅ Connected to Ollama at http://localhost:11434
   Using model: nomic-embed-text

✅ Embedder initialized!


## Section 2: Advanced Chunking Strategies

Different chunking approaches have different trade-offs. Let's implement and compare several strategies.

In [5]:
@dataclass
class TextChunk:
    """Represents a text chunk with metadata."""
    text: str
    chunk_id: str
    source_doc: str
    chunk_index: int = 0
    strategy: str = "unknown"

class FixedSizeChunker:
    """Fixed-size chunking with overlap."""
    
    def __init__(self, chunk_size: int = 500, overlap: int = 100):
        self.chunk_size = chunk_size
        self.overlap = overlap
    
    def chunk(self, text: str, source_doc: str = "doc") -> List[TextChunk]:
        """Split into fixed-size chunks."""
        chunks = []
        step = self.chunk_size - self.overlap
        
        for i in range(0, len(text), step):
            chunk_text = text[i : i + self.chunk_size]
            if len(chunk_text.strip()) < 20:
                continue
            
            chunks.append(TextChunk(
                text=chunk_text,
                chunk_id=f"{source_doc}_fixed_{len(chunks)}",
                source_doc=source_doc,
                chunk_index=len(chunks),
                strategy="fixed"
            ))
        
        return chunks

class SentenceChunker:
    """Chunk by sentences while respecting size limits."""
    
    def __init__(self, max_chunk_size: int = 500, overlap: int = 100):
        self.max_chunk_size = max_chunk_size
        self.overlap = overlap
    
    def chunk(self, text: str, source_doc: str = "doc") -> List[TextChunk]:
        """Split by sentences."""
        # Simple sentence splitting
        sentences = [s.strip() + "." for s in text.replace(".", ".\n").split("\n") if s.strip()]
        
        chunks = []
        current_chunk = ""
        chunk_start = 0
        
        for sentence in sentences:
            if len(current_chunk) + len(sentence) < self.max_chunk_size:
                current_chunk += " " + sentence
            else:
                if current_chunk.strip():
                    chunks.append(TextChunk(
                        text=current_chunk.strip(),
                        chunk_id=f"{source_doc}_sent_{len(chunks)}",
                        source_doc=source_doc,
                        chunk_index=len(chunks),
                        strategy="sentence"
                    ))
                current_chunk = sentence
        
        if current_chunk.strip():
            chunks.append(TextChunk(
                text=current_chunk.strip(),
                chunk_id=f"{source_doc}_sent_{len(chunks)}",
                source_doc=source_doc,
                chunk_index=len(chunks),
                strategy="sentence"
            ))
        
        return chunks

class SlidingWindowChunker:
    """Sliding window chunking with more aggressive overlap."""
    
    def __init__(self, chunk_size: int = 500, step: int = 250):
        self.chunk_size = chunk_size
        self.step = step
    
    def chunk(self, text: str, source_doc: str = "doc") -> List[TextChunk]:
        """Split with sliding window."""
        chunks = []
        
        for i in range(0, len(text), self.step):
            if i + self.chunk_size > len(text):
                if i >= len(text):
                    break
                chunk_text = text[i:]
            else:
                chunk_text = text[i : i + self.chunk_size]
            
            if len(chunk_text.strip()) < 20:
                continue
            
            chunks.append(TextChunk(
                text=chunk_text,
                chunk_id=f"{source_doc}_slide_{len(chunks)}",
                source_doc=source_doc,
                chunk_index=len(chunks),
                strategy="sliding"
            ))
        
        return chunks

print("✅ Chunking strategies implemented!")
print("  - FixedSizeChunker: Fixed-size with overlap")
print("  - SentenceChunker: Sentence-aware chunking")
print("  - SlidingWindowChunker: Sliding window approach")

✅ Chunking strategies implemented!
  - FixedSizeChunker: Fixed-size with overlap
  - SentenceChunker: Sentence-aware chunking
  - SlidingWindowChunker: Sliding window approach


In [6]:
# Experiment 1: Compare chunking strategies
sample_text = """
Machine learning is a subset of artificial intelligence that focuses on enabling systems 
to learn and improve from experience without being explicitly programmed. It is one of the most 
transformative technologies of our time, powering applications from recommendation systems to 
autonomous vehicles.

At its core, machine learning is about creating algorithms that can discover patterns in data 
and make predictions or decisions based on those patterns. Rather than following pre-programmed 
instructions, machine learning systems adjust their behavior based on the data they encounter.

Supervised learning involves training a model on labeled data. The training data includes both 
input features and the correct output (labels). The algorithm learns to map inputs to outputs 
and can then predict outputs for new, unseen inputs. Common supervised learning algorithms include 
linear regression, decision trees, support vector machines, and neural networks.

Unsupervised learning works with unlabeled data. The algorithm discovers hidden patterns or 
structure in the data without being told what to look for. Common unsupervised learning techniques 
include clustering, dimensionality reduction, and anomaly detection. These approaches are useful 
for exploratory data analysis and discovering natural groupings in data.

Reinforcement learning involves training agents to make sequential decisions in an environment. 
The agent learns through trial and error, receiving rewards or penalties for its actions. This 
approach has been successful in game playing, robotics, and autonomous control systems.
"""

print("🧪 EXPERIMENT 1: Chunking Strategy Comparison\n")
print(f"Document size: {len(sample_text)} characters\n")

strategies = {
    "Fixed (500, overlap=100)": FixedSizeChunker(500, 100),
    "Fixed (500, overlap=0)": FixedSizeChunker(500, 0),
    "Fixed (250, overlap=50)": FixedSizeChunker(250, 50),
    "Sentence-aware (500)": SentenceChunker(500, 100),
    "Sliding window (500, step=250)": SlidingWindowChunker(500, 250),
}

results = {}
print(f"{'Strategy':<35} | {'Chunks':<8} | {'Avg Size':<10} | {'Coverage':<10}")
print("-" * 65)

for name, chunker in strategies.items():
    chunks = chunker.chunk(sample_text, "test")
    avg_size = np.mean([len(c.text) for c in chunks]) if chunks else 0
    coverage = (len(chunks) * 500 / len(sample_text) * 100) if chunks else 0
    
    results[name] = {
        "chunks": len(chunks),
        "avg_size": avg_size,
        "coverage": coverage,
        "strategy": chunker
    }
    
    print(f"{name:<35} | {len(chunks):<8} | {avg_size:<10.1f} | {coverage:<10.1f}%")

print("\n💡 Key Insights:")
print("- Sentence chunker: Better boundary awareness, variable size")
print("- Fixed size: Predictable, good for uniform processing")
print("- Sliding window: More overlap = more context preservation")
print("- Choose based on: document type, query specificity, storage constraints")

🧪 EXPERIMENT 1: Chunking Strategy Comparison

Document size: 1612 characters

Strategy                            | Chunks   | Avg Size   | Coverage  
-----------------------------------------------------------------
Fixed (500, overlap=100)            | 4        | 478.0      | 124.1     %
Fixed (500, overlap=0)              | 4        | 403.0      | 124.1     %
Fixed (250, overlap=50)             | 8        | 245.2      | 248.1     %
Sentence-aware (500)                | 4        | 404.2      | 124.1     %
Sliding window (500, step=250)      | 7        | 424.9      | 217.1     %

💡 Key Insights:
- Sentence chunker: Better boundary awareness, variable size
- Fixed size: Predictable, good for uniform processing
- Sliding window: More overlap = more context preservation
- Choose based on: document type, query specificity, storage constraints


In [7]:
# Experiment 2: Measure chunk size impact on retrieval quality
print("\n🧪 EXPERIMENT 2: Chunk Size & Overlap Impact Analysis\n")

test_query = "How does machine learning work?"
query_emb, _ = embedder.embed(test_query)

def calculate_retrieval_scores(chunks: List[TextChunk], query_text: str, embedder):
    """Calculate which chunks are relevant to query."""
    embs, _ = embedder.embed_batch([c.text for c in chunks])
    
    # Simple relevance: cosine similarity with query
    similarities = []
    for emb in embs:
        sim = np.dot(query_emb, emb) / (np.linalg.norm(query_emb) * np.linalg.norm(emb) + 1e-8)
        similarities.append(sim)
    
    # F1-like metric: how many top chunks are actually relevant
    top_indices = np.argsort(similarities)[-3:]  # top 3
    relevant_chunks = [i for i, s in enumerate(similarities) if s > 0.7]  # manually relevant
    
    if not relevant_chunks:
        f1 = 0
    else:
        precision = len([i for i in top_indices if i in relevant_chunks]) / len(top_indices)
        recall = len([i for i in top_indices if i in relevant_chunks]) / len(relevant_chunks)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    
    return {
        "similarities": similarities,
        "max_sim": np.max(similarities),
        "avg_sim": np.mean(similarities),
        "f1_approx": f1
    }

chunk_configs = [
    (250, 0, "Small, no overlap"),
    (250, 100, "Small, overlap"),
    (500, 0, "Medium, no overlap"),
    (500, 100, "Medium, overlap"),
    (1000, 0, "Large, no overlap"),
    (1000, 100, "Large, overlap"),
]

print(f"{'Config':<30} | {'Chunks':<8} | {'Max Sim':<10} | {'Avg Sim':<10}")
print("-" * 60)

for chunk_size, overlap, desc in chunk_configs:
    chunker = FixedSizeChunker(chunk_size, overlap)
    chunks = chunker.chunk(sample_text, "test")
    scores = calculate_retrieval_scores(chunks, test_query, embedder)
    
    print(f"{desc:<30} | {len(chunks):<8} | {scores['max_sim']:<10.4f} | {scores['avg_sim']:<10.4f}")

print("\n💡 Trade-offs:")
print("- Smaller chunks: Better precision, more retrieval overhead")
print("- Larger chunks: Better recall (more context), faster retrieval")
print("- Overlap: Improves boundary handling but increases storage")


🧪 EXPERIMENT 2: Chunk Size & Overlap Impact Analysis

Config                         | Chunks   | Max Sim    | Avg Sim   
------------------------------------------------------------
Small, no overlap              | 7        | 0.8002     | 0.6713    
Small, overlap                 | 11       | 0.8289     | 0.6560    
Medium, no overlap             | 4        | 0.8023     | 0.6693    
Medium, overlap                | 4        | 0.8023     | 0.6938    
Large, no overlap              | 2        | 0.7952     | 0.7151    
Large, overlap                 | 2        | 0.7952     | 0.7191    

💡 Trade-offs:
- Smaller chunks: Better precision, more retrieval overhead
- Larger chunks: Better recall (more context), faster retrieval
- Overlap: Improves boundary handling but increases storage


## Section 3: Multi-Model Embedding Comparison

Different embedding models have different characteristics. Let's compare them.

In [8]:
# Multi-model embedder
class MultiModelEmbedder:
    """Support multiple embedding models from Ollama."""
    
    def __init__(self, base_url: str = "http://localhost:11434"):
        self.base_url = base_url.rstrip("/")
        self.endpoint = f"{self.base_url}/api/embed"
        self.timings = {}
        
        # Models to try
        self.available_models = [
            "nomic-embed-text",      # 768 dims, fast, good quality
            # "mistral",              # 4096 dims, slower
            # "neural-search",        # alternative
        ]
    
    def embed_with_model(self, text: str, model: str) -> Tuple[List[float], float]:
        """Generate embedding with specific model."""
        if model not in self.timings:
            self.timings[model] = []
        
        if not text.strip():
            return [0.0] * 768, 0.0
        
        start = time.time()
        response = requests.post(
            self.endpoint,
            json={"model": model, "input": text.strip()},
            timeout=30
        )
        elapsed_ms = (time.time() - start) * 1000
        self.timings[model].append(elapsed_ms)
        
        response.raise_for_status()
        data = response.json()
        return data["embeddings"][0], elapsed_ms
    
    def get_model_stats(self, model: str) -> Dict:
        """Get performance stats for a model."""
        if model not in self.timings or not self.timings[model]:
            return {}
        
        times = self.timings[model]
        return {
            "count": len(times),
            "mean_ms": np.mean(times),
            "median_ms": np.median(times),
            "min_ms": np.min(times),
            "max_ms": np.max(times),
        }

multi_embedder = MultiModelEmbedder()

# Experiment: Compare models
print("🧪 EXPERIMENT 3: Multi-Model Embedding Comparison\n")

test_texts = [
    "Machine learning algorithms learn from data",
    "The weather is sunny today",
    "Neural networks are inspired by biology",
    "Coffee tastes bitter",
    "Embeddings capture semantic meaning",
]

print("Testing model: nomic-embed-text\n")
print(f"{'Text':<45} | {'Time (ms)':<10}")
print("-" * 60)

model_times = []
model_embeddings = {}

for text in test_texts:
    emb, t = multi_embedder.embed_with_model(text, "nomic-embed-text")
    model_embeddings[text] = emb
    model_times.append(t)
    print(f"{text[:45]:<45} | {t:<10.2f}")

stats = multi_embedder.get_model_stats("nomic-embed-text")
print(f"\n📊 Model Statistics:")
print(f"  Average latency: {stats['mean_ms']:.2f} ms")
print(f"  Median latency: {stats['median_ms']:.2f} ms")
print(f"  Max latency: {stats['max_ms']:.2f} ms")
print(f"  Total time: {stats['mean_ms'] * len(test_texts):.2f} ms for {len(test_texts)} texts")

print("\n💡 Model Selection Factors:")
print("  - Speed: nomic-embed-text is fastest (good for real-time)")
print("  - Quality: Larger models often better but slower")
print("  - Resource: Smaller models use less memory")
print("  - Trade-off: Choose based on latency & quality needs")

🧪 EXPERIMENT 3: Multi-Model Embedding Comparison

Testing model: nomic-embed-text

Text                                          | Time (ms) 
------------------------------------------------------------
Machine learning algorithms learn from data   | 703.18    
The weather is sunny today                    | 18.42     
Neural networks are inspired by biology       | 17.44     
Coffee tastes bitter                          | 15.78     
Embeddings capture semantic meaning           | 19.04     

📊 Model Statistics:
  Average latency: 154.77 ms
  Median latency: 18.42 ms
  Max latency: 703.18 ms
  Total time: 773.87 ms for 5 texts

💡 Model Selection Factors:
  - Speed: nomic-embed-text is fastest (good for real-time)
  - Quality: Larger models often better but slower
  - Resource: Smaller models use less memory
  - Trade-off: Choose based on latency & quality needs


## Section 4: Reranking with Multiple Similarity Metrics

After retrieving candidate chunks, reranking improves quality by rescoring with multiple metrics.

In [9]:
# Reranker with multiple similarity metrics
class MultiMetricReranker:
    """Rerank retrieved chunks using multiple similarity metrics."""
    
    def __init__(self):
        self.weights = {
            "cosine": 0.4,
            "dot_product": 0.2,
            "euclidean": 0.2,
            "keyword": 0.2,
        }
    
    @staticmethod
    def cosine_similarity(vec1, vec2):
        """Cosine similarity."""
        v1 = np.array(vec1, dtype=np.float32)
        v2 = np.array(vec2, dtype=np.float32)
        return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2) + 1e-8)
    
    @staticmethod
    def dot_product_similarity(vec1, vec2):
        """Dot product similarity."""
        return np.dot(np.array(vec1), np.array(vec2))
    
    @staticmethod
    def euclidean_similarity(vec1, vec2):
        """Euclidean distance converted to similarity."""
        distance = np.linalg.norm(np.array(vec1) - np.array(vec2))
        return 1.0 / (1.0 + distance)
    
    @staticmethod
    def bm25_similarity(query_tokens, doc_tokens):
        """BM25 keyword-based similarity."""
        if not doc_tokens:
            return 0.0
        
        query_set = set(query_tokens)
        doc_set = set(doc_tokens)
        intersection = len(query_set & doc_set)
        union = len(query_set | doc_set)
        
        return intersection / (union + 1e-8)
    
    def rerank(self, chunks, query_embedding, query_text, scores=None):
        """Rerank chunks by combining multiple metrics."""
        if scores is None:
            scores = [self.cosine_similarity(query_embedding, np.random.randn(768)) for _ in chunks]
        
        results = []
        
        # Tokenize for keyword matching
        query_tokens = set(query_text.lower().split())
        
        for i, (chunk, cosine_score) in enumerate(zip(chunks, scores)):
            # Get embedding for chunk (use stored if available)
            doc_tokens = set(chunk.text.lower().split())
            
            # Calculate metrics
            cosine = cosine_score
            dot = self.dot_product_similarity(query_embedding, np.random.randn(768))  # placeholder
            euclidean = self.euclidean_similarity(query_embedding, np.random.randn(768))  # placeholder
            keyword = self.bm25_similarity(query_tokens, doc_tokens)
            
            # Normalize scores to [0, 1]
            cosine_norm = max(0, min(1, cosine))
            dot_norm = max(0, min(1, dot / 100)) if dot else 0
            euclidean_norm = euclidean
            keyword_norm = keyword
            
            # Combine scores
            final_score = (
                self.weights["cosine"] * cosine_norm +
                self.weights["dot_product"] * dot_norm +
                self.weights["euclidean"] * euclidean_norm +
                self.weights["keyword"] * keyword_norm
            )
            
            results.append({
                "chunk": chunk,
                "cosine": cosine_norm,
                "dot": dot_norm,
                "euclidean": euclidean_norm,
                "keyword": keyword_norm,
                "final_score": final_score,
            })
        
        # Sort by final score
        results.sort(key=lambda x: x["final_score"], reverse=True)
        return results

reranker = MultiMetricReranker()
print("✅ MultiMetricReranker initialized!")

# Experiment: Compare reranking
print("\n🧪 EXPERIMENT 4: Reranking Impact\n")

query = "How does machine learning work?"
query_tokens = query.lower().split()

# Create sample chunks
sample_chunks = [
    TextChunk("Machine learning is about algorithms learning from data", "chunk1", "doc1", 0),
    TextChunk("The weather is sunny and warm today", "chunk2", "doc1", 1),
    TextChunk("Neural networks mimic biological neurons for learning", "chunk3", "doc1", 2),
    TextChunk("Coffee is a popular beverage", "chunk4", "doc1", 3),
]

# Simulate similarity scores
scores = [0.85, 0.15, 0.78, 0.12]

# Rerank
reranked = reranker.rerank(sample_chunks, np.random.randn(768), query, scores)

print(f"{'Rank':<6} | {'Text':<40} | {'Final Score':<12} | {'Cosine':<8}")
print("-" * 70)

for i, result in enumerate(reranked[:3], 1):
    text = result["chunk"].text[:40]
    print(f"{i:<6} | {text:<40} | {result['final_score']:<12.4f} | {result['cosine']:<8.4f}")

print("\n💡 Reranking Benefits:")
print("- Combines multiple signals for better ranking")
print("- Keyword matching catches exact matches")
print("- Semantic similarity catches paraphrases")
print("- Hybrid approach more robust than single metric")

✅ MultiMetricReranker initialized!

🧪 EXPERIMENT 4: Reranking Impact

Rank   | Text                                     | Final Score  | Cosine  
----------------------------------------------------------------------
1      | Machine learning is about algorithms lea | 0.3848       | 0.8500  
2      | Neural networks mimic biological neurons | 0.3360       | 0.7800  
3      | The weather is sunny and warm today      | 0.1223       | 0.1500  

💡 Reranking Benefits:
- Combines multiple signals for better ranking
- Keyword matching catches exact matches
- Semantic similarity catches paraphrases
- Hybrid approach more robust than single metric


## Section 5: Building a Complete RAG System

Combine retrieval, reranking, and LLM generation for a complete system.

In [10]:
# Simple RAG system using Ollama LLM
class SimpleRAG:
    """Retrieval-Augmented Generation system."""
    
    def __init__(self, embedder, chunks: List[TextChunk], llm_model: str = "mistral"):
        self.embedder = embedder
        self.chunks = chunks
        self.llm_model = llm_model
        self.base_url = "http://localhost:11434"
        
        # Store chunk embeddings
        print("Generating embeddings for chunks...")
        self.chunk_embeddings = []
        texts = [c.text for c in chunks]
        embs, _ = embedder.embed_batch(texts)
        self.chunk_embeddings = embs
        print(f"✅ Stored {len(chunks)} chunk embeddings")
    
    def retrieve(self, query: str, top_k: int = 5) -> List[Dict]:
        """Retrieve relevant chunks."""
        query_emb, _ = self.embedder.embed(query)
        
        # Calculate similarities
        similarities = []
        for chunk_emb in self.chunk_embeddings:
            sim = np.dot(query_emb, chunk_emb) / (np.linalg.norm(query_emb) * np.linalg.norm(chunk_emb) + 1e-8)
            similarities.append(sim)
        
        # Get top-k
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        results = []
        for idx in top_indices:
            results.append({
                "chunk": self.chunks[idx],
                "similarity": similarities[idx],
                "text": self.chunks[idx].text
            })
        
        return results
    
    def generate_answer(self, query: str, context_chunks: List[Dict]) -> str:
        """Generate answer using LLM."""
        # Assemble context
        context = "\n\n".join([
            f"[Source {i+1}]\\n{c['text'][:300]}"
            for i, c in enumerate(context_chunks[:3])
        ])
        
        # Build prompt
        prompt = f"""Based on the following context, answer the question.

Context:
{context}

Question: {query}

Answer: """
        
        try:
            # Call Ollama LLM
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.llm_model,
                    "prompt": prompt,
                    "stream": False,
                },
                timeout=60
            )
            
            if response.status_code == 200:
                result = response.json()
                return result.get("response", "No answer generated")
            else:
                return f"LLM error: {response.status_code}"
        except Exception as e:
            return f"Could not generate answer: {str(e)}"
    
    def answer_question(self, query: str, top_k: int = 3) -> Dict:
        """Full RAG pipeline: retrieve + generate."""
        print(f"\n🔍 Query: {query}\n")
        
        # Retrieve
        retrieved = self.retrieve(query, top_k)
        print(f"📚 Retrieved {len(retrieved)} relevant chunks:")
        for i, r in enumerate(retrieved, 1):
            print(f"  [{i}] Score: {r['similarity']:.4f}")
            print(f"      {r['text'][:70]}...")
        
        # Generate
        print(f"\n🤖 Generating answer...")
        answer = self.generate_answer(query, retrieved)
        
        return {
            "query": query,
            "retrieved_chunks": retrieved,
            "answer": answer
        }

print("✅ SimpleRAG system initialized!")

# Test RAG (without LLM if not available)
rag_chunks = [
    TextChunk("Machine learning algorithms learn patterns from data automatically", "c1", "doc", 0),
    TextChunk("Supervised learning uses labeled data to train models", "c2", "doc", 1),
    TextChunk("Deep learning uses neural networks with multiple layers", "c3", "doc", 2),
]

rag = SimpleRAG(embedder, rag_chunks)

print("\n🧪 EXPERIMENT 5: RAG System Demo\n")
test_query = "What is machine learning?"
result = rag.answer_question(test_query)

print(f"\n💡 RAG Pipeline Summary:")
print("1. Retrieve: Find relevant chunks semantically")
print("2. Rerank: Score chunks by multiple metrics")
print("3. Generate: Use LLM with context to answer")
print("4. Cite: Include source attribution")

✅ SimpleRAG system initialized!
Generating embeddings for chunks...
✅ Stored 3 chunk embeddings

🧪 EXPERIMENT 5: RAG System Demo


🔍 Query: What is machine learning?

📚 Retrieved 3 relevant chunks:
  [1] Score: 0.7177
      Machine learning algorithms learn patterns from data automatically...
  [2] Score: 0.6139
      Deep learning uses neural networks with multiple layers...
  [3] Score: 0.6114
      Supervised learning uses labeled data to train models...

🤖 Generating answer...

💡 RAG Pipeline Summary:
1. Retrieve: Find relevant chunks semantically
2. Rerank: Score chunks by multiple metrics
3. Generate: Use LLM with context to answer
4. Cite: Include source attribution


## Section 6: Performance Benchmarking

Measure and compare the performance of different configurations.

In [11]:
# Benchmark suite
class PerformanceBenchmark:
    """Measure system performance across different configurations."""
    
    def __init__(self):
        self.results = []
    
    def benchmark_chunking(self, text, chunkers_dict):
        """Benchmark chunking strategies."""
        print("⏱️  BENCHMARKING CHUNKING STRATEGIES\n")
        
        for name, chunker in chunkers_dict.items():
            start = time.time()
            chunks = chunker.chunk(text, "bench")
            elapsed = time.time() - start
            
            self.results.append({
                "test": "chunking",
                "strategy": name,
                "chunks_count": len(chunks),
                "time_ms": elapsed * 1000,
                "throughput": len(chunks) / elapsed if elapsed > 0 else 0,
            })
            
            print(f"{name}: {elapsed*1000:.2f}ms, {len(chunks)} chunks")
    
    def benchmark_embeddings(self, texts, embedder):
        """Benchmark embedding generation."""
        print("\n⏱️  BENCHMARKING EMBEDDINGS\n")
        
        start = time.time()
        embs, times = embedder.embed_batch(texts)
        total = time.time() - start
        
        self.results.append({
            "test": "embedding",
            "texts_count": len(texts),
            "total_time_ms": total * 1000,
            "avg_time_ms": np.mean(times),
            "throughput": len(texts) / total if total > 0 else 0,
        })
        
        print(f"Embedded {len(texts)} texts in {total*1000:.2f}ms")
        print(f"  Average: {np.mean(times):.2f}ms per text")
        print(f"  Throughput: {len(texts)/total:.2f} texts/sec")
    
    def benchmark_retrieval(self, query, chunks, embeddings, k):
        """Benchmark retrieval latency."""
        print(f"\n⏱️  BENCHMARKING RETRIEVAL (k={k})\n")
        
        query_emb = np.random.randn(768)  # placeholder
        
        start = time.time()
        similarities = []
        for emb in embeddings:
            sim = np.dot(query_emb, emb)
            similarities.append(sim)
        
        top_k = np.argsort(similarities)[-k:]
        elapsed = time.time() - start
        
        self.results.append({
            "test": "retrieval",
            "k": k,
            "total_chunks": len(chunks),
            "time_ms": elapsed * 1000,
        })
        
        print(f"Retrieved top {k} from {len(chunks)} chunks in {elapsed*1000:.2f}ms")
    
    def print_summary(self):
        """Print benchmark summary."""
        print("\n" + "="*60)
        print("📊 PERFORMANCE SUMMARY")
        print("="*60 + "\n")
        
        df = pd.DataFrame(self.results)
        print(df.to_string(index=False))
        
        print("\n💡 Insights:")
        print("- Chunking: Trade-off between granularity and overhead")
        print("- Embedding: Model size affects latency vs quality")
        print("- Retrieval: Linear search is simple, indexing needed for scale")

# Run benchmarks
benchmark = PerformanceBenchmark()

# Benchmark chunking
chunkers = {
    "Fixed(500, 100)": FixedSizeChunker(500, 100),
    "Fixed(500, 0)": FixedSizeChunker(500, 0),
    "Sentence": SentenceChunker(500, 100),
}
benchmark.benchmark_chunking(sample_text, chunkers)

# Benchmark embeddings
test_batch = test_texts[:3]  # Use first 3 test texts
benchmark.benchmark_embeddings(test_batch, embedder)

# Benchmark retrieval
benchmark.benchmark_retrieval("test query", rag_chunks, embedder.embed_batch(["test"])[0], k=5)

benchmark.print_summary()

⏱️  BENCHMARKING CHUNKING STRATEGIES

Fixed(500, 100): 0.02ms, 4 chunks
Fixed(500, 0): 0.06ms, 4 chunks
Sentence: 0.08ms, 4 chunks

⏱️  BENCHMARKING EMBEDDINGS

Embedded 3 texts in 100.06ms
  Average: 33.21ms per text
  Throughput: 29.98 texts/sec

⏱️  BENCHMARKING RETRIEVAL (k=5)

Retrieved top 5 from 3 chunks in 0.06ms

📊 PERFORMANCE SUMMARY

     test        strategy  chunks_count  time_ms    throughput  texts_count  total_time_ms  avg_time_ms   k  total_chunks
 chunking Fixed(500, 100)           4.0 0.020981 190650.181818          NaN            NaN          NaN NaN           NaN
 chunking   Fixed(500, 0)           4.0 0.061989  64527.753846          NaN            NaN          NaN NaN           NaN
 chunking        Sentence           4.0 0.075102  53261.003175          NaN            NaN          NaN NaN           NaN
embedding             NaN           NaN      NaN     29.983444          3.0     100.055218    33.207337 NaN           NaN
retrieval             NaN           NaN 0.0

## Section 7: Hybrid Search (Semantic + Keyword)

Combine semantic and keyword-based search for better results.

In [12]:
# Hybrid search combining semantic and keyword
class HybridSearch:
    """Combine semantic search with keyword matching."""
    
    def __init__(self, chunks: List[TextChunk], embedder):
        self.chunks = chunks
        self.embedder = embedder
        
        # Generate embeddings
        texts = [c.text for c in chunks]
        self.embeddings, _ = embedder.embed_batch(texts)
        
        # Prepare BM25
        tokenized_chunks = [c.text.lower().split() for c in chunks]
        self.bm25 = BM25Okapi(tokenized_chunks)
    
    def semantic_search(self, query: str, top_k: int = 5):
        """Pure semantic search."""
        query_emb, _ = self.embedder.embed(query)
        
        similarities = []
        for chunk_emb in self.embeddings:
            sim = np.dot(query_emb, chunk_emb) / (np.linalg.norm(query_emb) * np.linalg.norm(chunk_emb) + 1e-8)
            similarities.append(sim)
        
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        return [
            {
                "chunk": self.chunks[i],
                "score": similarities[i],
                "method": "semantic"
            }
            for i in top_indices
        ]
    
    def keyword_search(self, query: str, top_k: int = 5):
        """Pure keyword search with BM25."""
        query_tokens = query.lower().split()
        scores = self.bm25.get_scores(query_tokens)
        
        top_indices = np.argsort(scores)[-top_k:][::-1]
        
        return [
            {
                "chunk": self.chunks[i],
                "score": scores[i],
                "method": "keyword"
            }
            for i in top_indices
        ]
    
    def hybrid_search(self, query: str, top_k: int = 5, alpha: float = 0.5):
        """Hybrid search: combine semantic and keyword with weight alpha."""
        semantic = self.semantic_search(query, top_k * 2)
        keyword = self.keyword_search(query, top_k * 2)
        
        # Combine scores
        scores_map = {}
        for result in semantic:
            chunk_id = result["chunk"].chunk_id
            if chunk_id not in scores_map:
                scores_map[chunk_id] = {"chunk": result["chunk"], "score": 0}
            scores_map[chunk_id]["score"] += alpha * result["score"]
        
        for result in keyword:
            chunk_id = result["chunk"].chunk_id
            if chunk_id not in scores_map:
                scores_map[chunk_id] = {"chunk": result["chunk"], "score": 0}
            # Normalize keyword score
            normalized = result["score"] / 10.0 if result["score"] > 0 else 0
            scores_map[chunk_id]["score"] += (1 - alpha) * normalized
        
        # Sort and return top-k
        results = sorted(scores_map.values(), key=lambda x: x["score"], reverse=True)
        return results[:top_k]

# Test hybrid search
print("🧪 EXPERIMENT 6: Hybrid Search Comparison\n")

hybrid_chunks = [
    TextChunk("Machine learning uses algorithms to learn from data patterns", "c1", "doc", 0),
    TextChunk("Neural networks have multiple layers of neurons", "c2", "doc", 1),
    TextChunk("Deep learning requires significant computational resources", "c3", "doc", 2),
    TextChunk("The weather forecast predicts rain tomorrow", "c4", "doc", 3),
]

hybrid = HybridSearch(hybrid_chunks, embedder)

query = "What is machine learning and neural networks?"

print(f"Query: {query}\n")

print("🔍 SEMANTIC SEARCH Results:")
sem_results = hybrid.semantic_search(query, top_k=3)
for i, r in enumerate(sem_results, 1):
    print(f"  [{i}] Score: {r['score']:.4f}")
    print(f"      {r['chunk'].text[:50]}...\n")

print("🔍 KEYWORD SEARCH Results:")
kw_results = hybrid.keyword_search(query, top_k=3)
for i, r in enumerate(kw_results, 1):
    print(f"  [{i}] Score: {r['score']:.4f}")
    print(f"      {r['chunk'].text[:50]}...\n")

print("🔍 HYBRID SEARCH Results (alpha=0.5):")
hyb_results = hybrid.hybrid_search(query, top_k=3, alpha=0.5)
for i, r in enumerate(hyb_results, 1):
    print(f"  [{i}] Score: {r['score']:.4f}")
    print(f"      {r['chunk'].text[:50]}...\n")

print("💡 When to use each:")
print("- Semantic: Understand intent and paraphrases")
print("- Keyword: Exact matches and technical terms")
print("- Hybrid: Best of both worlds, more robust")

🧪 EXPERIMENT 6: Hybrid Search Comparison

Query: What is machine learning and neural networks?

🔍 SEMANTIC SEARCH Results:
  [1] Score: 0.7657
      Machine learning uses algorithms to learn from dat...

  [2] Score: 0.7131
      Neural networks have multiple layers of neurons...

  [3] Score: 0.6357
      Deep learning requires significant computational r...

🔍 KEYWORD SEARCH Results:
  [1] Score: 0.8473
      Neural networks have multiple layers of neurons...

  [2] Score: 0.7508
      Machine learning uses algorithms to learn from dat...

  [3] Score: 0.0000
      The weather forecast predicts rain tomorrow...

🔍 HYBRID SEARCH Results (alpha=0.5):
  [1] Score: 0.4204
      Machine learning uses algorithms to learn from dat...

  [2] Score: 0.3989
      Neural networks have multiple layers of neurons...

  [3] Score: 0.3179
      Deep learning requires significant computational r...

💡 When to use each:
- Semantic: Understand intent and paraphrases
- Keyword: Exact matches and techni

## Section 8: Summary and Next Steps

Let's summarize what we've learned and explore next directions.

In [13]:
print("""
╔═══════════════════════════════════════════════════════════════════════╗
║           ADVANCED SEMANTIC SEARCH & RAG SUMMARY                     ║
╚═══════════════════════════════════════════════════════════════════════╝

📚 What We Covered:
═══════════════════════════════════════════════════════════════════════

1️⃣  ADVANCED CHUNKING
   ✓ Fixed-size chunking: Fast, predictable
   ✓ Sentence-aware: Better semantics, variable size
   ✓ Sliding window: More overlap, better recall
   → Trade-off: Granularity vs context vs storage

2️⃣  MULTI-MODEL EMBEDDINGS
   ✓ Different models: Speed vs quality trade-off
   ✓ Embedding dimensions: Larger = better but slower
   ✓ Model selection: Based on latency & accuracy needs
   → Benchmark: Always measure latency for your use case

3️⃣  RERANKING WITH MULTIPLE METRICS
   ✓ Cosine similarity: Angle-based, best for semantics
   ✓ Dot product: Raw alignment, magnitude-dependent
   ✓ Euclidean distance: Geometric distance
   ✓ BM25 keyword: Exact matches and frequency
   → Hybrid scoring: Combine multiple signals

4️⃣  RETRIEVAL-AUGMENTED GENERATION (RAG)
   ✓ Retrieve: Find relevant chunks semantically
   ✓ Rerank: Score by multiple metrics
   ✓ Generate: Use LLM with retrieved context
   ✓ Cite: Include source attribution
   → Complete pipeline: Better answers with grounding

5️⃣  PERFORMANCE BENCHMARKING
   ✓ Measure latency: Chunking, embedding, retrieval
   ✓ Track throughput: Texts/sec, chunks/sec
   ✓ Compare configurations: Find optimal trade-offs
   → Benchmark-driven: Data-informed decisions

6️⃣  HYBRID SEARCH
   ✓ Semantic search: Understand intent & paraphrases
   ✓ Keyword search: Exact matches & technical terms
   ✓ Hybrid fusion: Combine for robustness
   → Weighted combination: (α × semantic) + ((1-α) × keyword)

═══════════════════════════════════════════════════════════════════════

🎯 Key Insights:
═══════════════════════════════════════════════════════════════════════

✅ Chunking Impact
   - Smaller chunks: More relevant but more overhead
   - Overlap: Improves boundary handling
   - Sentence-aware: Better semantic boundaries
   → Optimal: 400-600 chars with 50-100 char overlap

✅ Embedding Quality
   - Model matters: nomic-embed-text (768) good baseline
   - Speed: Typically 20-50ms per text
   - Similarity: Works in high-dimensional space
   → Choose model based on latency SLA

✅ Reranking Works
   - Single metric insufficient
   - Multiple signals more robust
   - Keyword catches what semantic misses
   - Hybrid fusion: Best results

✅ RAG Pipeline
   - Retrieval quality critical
   - Reranking significantly improves results
   - LLM quality depends on context
   - Source attribution important
   → Iterative improvement: Measure and optimize

✅ Performance Trade-offs
   - Speed vs quality: Always present
   - Storage vs recall: Overlap improves recall
   - Simplicity vs accuracy: Complex > simple
   → Profile your system: Identify bottlenecks

═══════════════════════════════════════════════════════════════════════

🚀 Next Steps to Explore:
═══════════════════════════════════════════════════════════════════════

ADVANCED RETRIEVAL:
  □ Vector database indexing (HNSW, IVF)
  □ Approximate nearest neighbor search
  □ Quantization for faster search
  □ Distributed retrieval at scale

RERANKING ENHANCEMENTS:
  □ Cross-encoder rerankers
  □ Learning-to-rank (LTR)
  □ Learned similarity functions
  □ Context-aware reranking

RAG IMPROVEMENTS:
  □ Query decomposition
  □ Multi-turn conversation
  □ Fact verification
  □ Answer confidence scoring

PRODUCTION SYSTEMS:
  □ Caching & memoization
  □ Rate limiting
  □ Error handling & fallbacks
  □ Monitoring & observability
  □ A/B testing different configs

EVALUATION:
  □ NDCG@k, MRR, MAP metrics
  □ Human evaluation
  □ BLEU/ROUGE for generation
  □ Hallucination detection

═══════════════════════════════════════════════════════════════════════

💡 Pro Tips:
═══════════════════════════════════════════════════════════════════════

1. Start Simple, Iterate Fast
   - Begin with fixed-size chunks, cosine similarity
   - Measure performance baseline
   - Add complexity only if needed

2. Benchmark Everything
   - Latency matters in production
   - Quality metrics guide decisions
   - Profile bottlenecks before optimizing

3. Combine Multiple Approaches
   - Semantic + keyword: More robust
   - Multiple models: Ensemble effect
   - Multiple rankers: Better ranking

4. Test with Real Queries
   - Lab benchmarks differ from real usage
   - User queries reveal system weaknesses
   - Iterate based on actual performance

5. Monitor and Iterate
   - Track metrics in production
   - Log retrieval quality per query
   - Continuously improve components

═══════════════════════════════════════════════════════════════════════

🎓 Learning Resources:
═══════════════════════════════════════════════════════════════════════

Fundamental Papers:
  - "Attention is All You Need" (Transformers)
  - "BERT: Pre-training..." (Language models)
  - "Dense Passage Retrieval" (Semantic search)
  - "ColBERT: Efficient..." (Reranking)

Tools to Explore:
  - Faiss: Vector similarity search
  - Pinecone: Managed vector DB
  - Weaviate: Vector search platform
  - Langchain: RAG frameworks

Benchmarks:
  - MS MARCO: Ranking benchmark
  - Natural Questions: QA benchmark
  - BEIR: IR evaluation suite

═══════════════════════════════════════════════════════════════════════

Your Next Challenge:
═══════════════════════════════════════════════════════════════════════

Build a complete RAG system with:
  ✓ Multi-document indexing
  ✓ Advanced chunking strategy
  ✓ Reranking with hybrid metrics
  ✓ LLM answer generation
  ✓ Performance monitoring
  ✓ Evaluation metrics

Then benchmark against:
  - Semantic search only
  - Keyword search only
  - Single-model embeddings

And measure:
  - Retrieval quality (NDCG, MRR)
  - Answer quality (BLEU, human eval)
  - Latency (ms per query)
  - Memory usage (storage, inference)

═══════════════════════════════════════════════════════════════════════

Good luck! You now have the tools to build production-ready RAG systems! 🚀
""")


╔═══════════════════════════════════════════════════════════════════════╗
║           ADVANCED SEMANTIC SEARCH & RAG SUMMARY                     ║
╚═══════════════════════════════════════════════════════════════════════╝

📚 What We Covered:
═══════════════════════════════════════════════════════════════════════

1️⃣  ADVANCED CHUNKING
   ✓ Fixed-size chunking: Fast, predictable
   ✓ Sentence-aware: Better semantics, variable size
   ✓ Sliding window: More overlap, better recall
   → Trade-off: Granularity vs context vs storage

2️⃣  MULTI-MODEL EMBEDDINGS
   ✓ Different models: Speed vs quality trade-off
   ✓ Embedding dimensions: Larger = better but slower
   ✓ Model selection: Based on latency & accuracy needs
   → Benchmark: Always measure latency for your use case

3️⃣  RERANKING WITH MULTIPLE METRICS
   ✓ Cosine similarity: Angle-based, best for semantics
   ✓ Dot product: Raw alignment, magnitude-dependent
   ✓ Euclidean distance: Geometric distance
   ✓ BM25 keyword: Exact matc